In [1]:
import os

from pdbfixer import PDBFixer
import openmm as mm
from openmm import unit
from openmm import app
import nglview as nv

## SER analysis

Let's analyze a particular feature in a SER residue in the Barnase-Barstar complex. As we can see in the PDB, there are two possibles configurations for this SER, labeled as ASER and BSER, despite having the same atoms, is in the positions of these where the difference relies. 

![title](./ser.png)

The question is, if we edit the PDB file with pdbfixer, which configuration does the tool will take, ASER or BSER?

In the image we can identify the chain id (D) and residue id (14), so let's start isolating it.

In [2]:
fixer = PDBFixer(filename='1brs.pdb')

In [3]:
fixer.topology

<Topology; 12 chains, 1101 residues, 5151 atoms, 4740 bonds>

In [4]:
bad_chains = []
for chain in fixer.topology.chains():
    if chain.id != 'D':
        bad_chains.append(chain.index)

In [5]:
bad_chains

[0, 1, 2, 4, 5, 6, 7, 8, 10, 11]

In [6]:
fixer.removeChains(bad_chains)

In [7]:
fixer.topology

<Topology; 2 chains, 164 residues, 770 atoms, 706 bonds>

In [8]:
for chain in fixer.topology.chains():
    for residue in fixer.topology.residues():
        if residue.name == 'SER':
            print(chain.id, chain.index, residue.id, residue.name)

D 0 12 SER
D 0 14 SER
D 0 59 SER
D 0 69 SER
D 0 89 SER
D 1 12 SER
D 1 14 SER
D 1 59 SER
D 1 69 SER
D 1 89 SER


With this output we can verify that D chains were obtain.

<div class="alert alert-info">
<strong>Question:</strong> There was two ways I tried to print serine residues; first, I declared residue in chain.residues(), but this only print one chain, why fixer.topology.residues() show both D chains?
</div>

In [9]:
x = 0
for chain in fixer.topology.chains():
    for residue in fixer.topology.residues():
        for atom in residue.atoms():
            if residue.name == 'SER':
                x = x+1;
                print(chain.index, chain.id, residue.id, residue.name, atom.id, atom.name)
print("Total atomos en SER: "+ str(x))

0 D 12 SER 2672 N
0 D 12 SER 2673 CA
0 D 12 SER 2674 C
0 D 12 SER 2675 O
0 D 12 SER 2676 CB
0 D 12 SER 2677 OG
0 D 14 SER 2686 N
0 D 14 SER 2687 CA
0 D 14 SER 2688 C
0 D 14 SER 2689 O
0 D 14 SER 2690 CB
0 D 14 SER 2692 OG
0 D 59 SER 3070 N
0 D 59 SER 3071 CA
0 D 59 SER 3072 C
0 D 59 SER 3073 O
0 D 59 SER 3074 CB
0 D 59 SER 3075 OG
0 D 69 SER 3122 N
0 D 69 SER 3123 CA
0 D 69 SER 3124 C
0 D 69 SER 3125 O
0 D 69 SER 3126 CB
0 D 69 SER 3127 OG
0 D 89 SER 3274 N
0 D 89 SER 3275 CA
0 D 89 SER 3276 C
0 D 89 SER 3277 O
0 D 89 SER 3278 CB
0 D 89 SER 3279 OG
1 D 12 SER 2672 N
1 D 12 SER 2673 CA
1 D 12 SER 2674 C
1 D 12 SER 2675 O
1 D 12 SER 2676 CB
1 D 12 SER 2677 OG
1 D 14 SER 2686 N
1 D 14 SER 2687 CA
1 D 14 SER 2688 C
1 D 14 SER 2689 O
1 D 14 SER 2690 CB
1 D 14 SER 2692 OG
1 D 59 SER 3070 N
1 D 59 SER 3071 CA
1 D 59 SER 3072 C
1 D 59 SER 3073 O
1 D 59 SER 3074 CB
1 D 59 SER 3075 OG
1 D 69 SER 3122 N
1 D 69 SER 3123 CA
1 D 69 SER 3124 C
1 D 69 SER 3125 O
1 D 69 SER 3126 CB
1 D 69 SER 3127 OG
1

A traves del residue.id y atom.id se puede confirmar que la configuracion que toma pdbfixer para la SER (14) es la A.

In [10]:
fixer.findMissingResidues()
missing_residues = fixer.missingResidues
print(f"{len(missing_residues)} missing residues")

fixer.findNonstandardResidues()
nonstandard_residues = fixer.nonstandardResidues
print(f"{len(nonstandard_residues)} non standard residues")

fixer.findMissingAtoms()
missing_atoms = fixer.missingAtoms
missing_terminals = fixer.missingTerminals
print(f"{len(missing_atoms)} missing atoms")
print(f"{len(missing_terminals)} missing terminals")

if len(nonstandard_residues)>0:
    fixer.replaceNonstandardResidues()

if len(missing_atoms)>0:
    fixer.addMissingAtoms()

1 missing residues
0 non standard residues
3 missing atoms
1 missing terminals


In [11]:
fixer.topology

<Topology; 2 chains, 166 residues, 795 atoms, 731 bonds>

In [12]:
modeller = app.Modeller(fixer.topology, fixer.positions)
app.PDBFile.writeFile(modeller.topology, modeller.positions, open('barnase-ser.pdb', 'w'))

In [13]:
x = 0
for chain in fixer.topology.chains():
    for residue in fixer.topology.residues():
        for atom in residue.atoms():
            if residue.name == 'SER':
                x = x+1;
                print(chain.index, chain.id, residue.id, residue.name, atom.id, atom.name)
print("Total atomos en SER: "+ str(x))

0 D 12 SER 88 N
0 D 12 SER 89 CA
0 D 12 SER 90 C
0 D 12 SER 91 O
0 D 12 SER 92 CB
0 D 12 SER 93 OG
0 D 14 SER 102 N
0 D 14 SER 103 CA
0 D 14 SER 104 C
0 D 14 SER 105 O
0 D 14 SER 106 CB
0 D 14 SER 107 OG
0 D 59 SER 486 N
0 D 59 SER 487 CA
0 D 59 SER 488 C
0 D 59 SER 489 O
0 D 59 SER 490 CB
0 D 59 SER 491 OG
0 D 69 SER 560 N
0 D 69 SER 561 CA
0 D 69 SER 562 C
0 D 69 SER 563 O
0 D 69 SER 564 CB
0 D 69 SER 565 OG
0 D 89 SER 712 N
0 D 89 SER 713 CA
0 D 89 SER 714 C
0 D 89 SER 715 O
0 D 89 SER 716 CB
0 D 89 SER 717 OG
0 D 89 SER 718 OXT
1 D 12 SER 88 N
1 D 12 SER 89 CA
1 D 12 SER 90 C
1 D 12 SER 91 O
1 D 12 SER 92 CB
1 D 12 SER 93 OG
1 D 14 SER 102 N
1 D 14 SER 103 CA
1 D 14 SER 104 C
1 D 14 SER 105 O
1 D 14 SER 106 CB
1 D 14 SER 107 OG
1 D 59 SER 486 N
1 D 59 SER 487 CA
1 D 59 SER 488 C
1 D 59 SER 489 O
1 D 59 SER 490 CB
1 D 59 SER 491 OG
1 D 69 SER 560 N
1 D 69 SER 561 CA
1 D 69 SER 562 C
1 D 69 SER 563 O
1 D 69 SER 564 CB
1 D 69 SER 565 OG
1 D 89 SER 712 N
1 D 89 SER 713 CA
1 D 89 SER 71

After editing, it can't be shown if SER maintain the A configuration.